In [1]:
import contextily as ctx
import geopandas as gpd
import kagglehub
import matplotlib.pyplot as plt
import numpy as np
import osmnx
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer

/home/kaarentenson/PythonV/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
DaytonDf = pd.read_csv("13/Dayton.csv")
roadNetwork = osmnx.io.load_graphml("data/road.graphml")
study_area = osmnx.geocode_to_gdf("Dayton")
# congestionData = pd.read_csv("data/DaytonCongestionsDf.csv")

In [8]:
DaytonDf["Start_Time"] = pd.to_datetime(
    DaytonDf["Start_Time"], format="mixed", utc=True
).dt.floor("h")
DaytonDf["End_Time"] = pd.to_datetime(
    DaytonDf["End_Time"], format="mixed", utc=True
).dt.floor("h")

In [10]:
start_times = DaytonDf["Start_Time"].dropna().unique()
print(len(start_times))
np.random.shuffle(start_times)
print(len(start_times.unique()))
train_times, test_times = train_test_split(start_times, test_size=0.2, random_state=42)
train_df = DaytonDf[DaytonDf["Start_Time"].isin(train_times)]
test_df = DaytonDf[DaytonDf["Start_Time"].isin(test_times)]

12495


/tmp/ipykernel_553162/539175408.py:3: UserWarning: you are shuffling a 'DatetimeArray' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(start_times)


12495


In [34]:
def normalizeColumns(df, normalizer: Normalizer = None):
    if normalizer == None:
        normalizer = Normalizer()
        normalizer.fit(df)
    return normalizer.transform(df), normalizer


def oneHotEncode(df, columns):
    pd.get_dummies(df, columns=columns)


def preProccessTestData(accidentDf, dates, normalizer: Normalizer, kmeans: KMeans):
    accidentDf["cluster"] = kmeans.predict(test_df[["Start_Lat", "Start_Lat"]])
    X_data = pd.DataFrame({"time": dates})
    X_data["temp"] = X_data["time"].apply(
        lambda x: accidentDf[accidentDf["Start_Time"] == x]["Temperature(F)"].mean()
    )
    y_data = pd.DataFrame()
    for i in range(kmeans.n_clusters):
        y_data[str(i)] = X_data["time"].apply(
            lambda x: accidentDf[
                (accidentDf["Start_Time"] == x) & (accidentDf["cluster"] == i)
            ].shape[0]
        )

    return X_data, y_data


def preProccessTrainData(
    accidentDf,
    dates,
    numberOfClusters=5,
):

    kmeans = KMeans(n_clusters=numberOfClusters)
    kmeans.fit(accidentDf[["Start_Lat", "Start_Lat"]])
    accidentDf["cluster"] = kmeans.labels_
    X_data = pd.DataFrame({"time": dates})
    X_data["temp"] = X_data["time"].apply(
        lambda x: accidentDf[accidentDf["Start_Time"] == x]["Temperature(F)"].mean()
    )
    y_data = pd.DataFrame()
    for i in range(numberOfClusters):
        y_data[str(i)] = X_data["time"].apply(
            lambda x: accidentDf[
                (accidentDf["Start_Time"] == x) & (accidentDf["cluster"] == i)
            ].shape[0]
        )
    X_data["temp"]=X_data["temp"].fillna(X_data["temp"].mean())
    print(X_data["temp"])
    X_data[["temp"]], normalizer =normalizeColumns(X_data[["temp"]])
    return X_data, y_data, normalizer, kmeans

In [35]:
train_X, train_y, normalizer, kmeans = preProccessTrainData(train_df, train_times)
test_X, test_y= preProccessTestData(test_df, test_times, normalizer, kmeans)

/tmp/ipykernel_553162/253582968.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  accidentDf["cluster"] = kmeans.labels_


0       62.000000
1       78.727273
2       63.000000
3       23.000000
4       54.633333
          ...    
9991    57.000000
9992    48.000000
9993    68.000000
9994    80.000000
9995    44.000000
Name: temp, Length: 9996, dtype: float64


/tmp/ipykernel_553162/253582968.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  accidentDf["cluster"] = kmeans.predict(test_df[["Start_Lat", "Start_Lat"]])


In [38]:
train_y

,0,1,2,3,4
0,3,0,0,0,0
1,11,0,0,0,0
2,1,0,0,0,0
3,2,0,0,0,0
4,5,1,0,0,0
...,...,...,...,...,...
9991,2,0,0,0,0
9992,1,0,0,0,0
9993,1,0,0,0,0
9994,2,0,0,0,0
